### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [1]:
library(tibble)

In [2]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


In [3]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [4]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [5]:
head(df,2)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,1050,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
df$flag <- FALSE
df$true_false <- FALSE


In [7]:
head(df, 2)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,1050,4.0700,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


## <font color = yellow>1. Create data & dfs

In [8]:
create_data <- function () {   
   
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$date <- as.Date(df$date, origin = "1899-12-30")
 
    df$flag <- FALSE
    df$true_false <- FALSE
    
    sale_records <- data.frame() 

    # create dfs from filtered df
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    b <- buys[1,]
    s <- sells[1,]
    
    
    # Assign variables to the global environment
    assign("df", df, envir = .GlobalEnv)
    assign("buys", buys, envir = .GlobalEnv)
    assign("sells", sells, envir = .GlobalEnv)
    assign("b", b, envir = .GlobalEnv)
    assign("s", s, envir = .GlobalEnv)
    
    }

create_data()


In [9]:
dim(df)


[1] 41 23

In [10]:
# boring b is just the next row from buys
t(b)

,1
index,1
date,2022-12-31
type,Buy
co,AMC
quan,1050
pps,4.07
basis,4273.5
sale_quan,0
flag,FALSE
buy_cf_quan,0


## <font color = lime> Examining data and dfs

In [47]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 3)
head(sells,3)

[1] 23

[1] 18

[1] 18 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,0,0.000,0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,5,2023-01-23,Buy,AMC,1000,5.788,5788,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,AMC,0,5.5124,5.5124,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,5.5124,5512.4000,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.2550,6255.0000,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


In [46]:
dim(b)
b

[1]  1 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,0,0,0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


### <font color = yellow>2. update_b() - updates boring b with corresponding values from an s row, .... and with any NEW c/f  
    
#### b>s, b<s, b==s,

In [13]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

for loop, loop is i:  1 
for loop, loop is i:  2 
for loop, loop is i:  3 


In [14]:
last_row <- tail(buys, n = 1)
#t(last_row)

#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [15]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function() {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) {
            
           
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            
             # populate the buy_cf_xyz
            b$buy_cf_quan <- (b$quan - s$quan)        
            b$buy_cf_pps <- b$pps
            b$buy_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            
            # Has buy c/f 
            b$flag <- TRUE         
            cat("update fct(), b$quan > s$quan ... new buy carryforward. flag = TRUE ", b$buy_cf_quan, "\n")
            
            
            
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
            
             cat("update fct(), s$quan > b$quan ... new sell carryforward. true_false = TRUE ", b$sell_quan, "\n")
            
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {            
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
             b$flag <- FALSE   
             b$true_false <- FALSE
          
 #           s$type <- "SOLD"  
            cat("update fct(), b$quan === s$quan ... No NEW carryforward. flag & true_false = FALSE ", b$sell_quan, "\n")
            
            }
            
        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

update fct(), b$quan > s$quan ... new buy carryforward. flag = TRUE  50 


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE


In [45]:
t(updated_b)

,1
index,1
date,2022-12-31
type,Buy CF
co,AMC
quan,0
pps,0
basis,0
sale_quan,0
flag,FALSE
buy_cf_quan,0


### <font color = yellow>3. create new sales_records (rbind) with the updated_b 

In [17]:
df_name <- "iris"  # Replace with your data frame name
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)
         

[1] TRUE

In [18]:
df_name <- "sale_records"
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

[1] FALSE

In [19]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    cat("Newly created sale_records with updated_b data. ", "\n")
    sale_records <- data.frame()
    sale_records <- rbind(sale_records, updated_b)    
}
sale_records

Newly created sale_records with updated_b data.  


index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE


In [20]:
t(sale_records)

index,1
date,2022-12-31
type,Buy
co,AMC
quan,1050
pps,4.07
basis,4273.5
sale_quan,1000
flag,TRUE
buy_cf_quan,50
buy_cf_pps,4.07


### <font color = yellow> 4. update buys & sells with the update_buysell_with_c/f () fct, 

##### <font color = yellow>  updates buys or sells with the new cf (when b > s or s > b ) c/f
    
##### <font color = lime> Uses add_row () to push c/f into next buys or sells row

#### <font color = lime> How it's done

In [21]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

x,y
<dbl>,<dbl>
1,3
4,0
2,2
3,1


In [22]:
first_row <- head(sells, 1)
first_row

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


In [23]:
dim(buys)
dim(sells)

[1] 23 23

[1] 18 23

In [24]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_buysell_withcf <- function(buys, sells) {

    #  Check the most recent row ??? of sale_record or the last row of sale_record (was bind() or add_row() ? )
    last_row <- tail(sale_records, n = 1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    # both start with the same inputs from the sale_records
    b1 <- last_row
    s1 <- last_row
    
    
    
    
    
    
    
    #  b > s  ... buy c/f will be created 
   
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        
        
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
        
        
        
   
        
        
# -------------------------------------------------------------------------------------------------------------------------   
        
        
        # NEW  ... This section makes changes in sells[2, ]  which becomes sells[3,]  after the add_row() executes
                
        
        s2cf_quan <- sells$quan[2] - s1$sold_cf_quan
        s2cf_pps <- sells$pps[2]
        
        cat("the s2cf_quan is: ", s2cf_quan, "\n")
        cat("the s2cf_pps is ", s2cf_pps, "\n")
        
        sells$quan[2] <- s2cf_quan
        sells$pps[2] <-  s1$sell_pps                     #    s2cf_pps 
        sells[2, ]
        sells$basis[2] <- sells$pps[2] * s2cf_quan
        
       
        
# -------------------------------------------------------------------------------------------------------------------------    
        
        
        #  The section updates b1   .... t/f buys
        
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- FALSE
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        test <- test %>% add_row(b1, .before = 1)
        buys <- test
        
  # ---------------------------------------------------------------------------------------------------------------      
        
               
        # this section updates s1 ... t/f sells
       
        
            
        last_row$co <- "c/f"    
        
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <-  s1$sell_pps               #    s1$sold_cf_pps
        s1$basis <- s1$sell_pps
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
        
 
        
        
# =================================================================================================================
# ==============================================================================================================
        
        
             #   s > b ... sell carryforward will be created    
        

    
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
        
        
       
        cat("\n", "There is Sold carryforward. Update sells ", last_row$sold_cf_quan, "\n")
           # cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/f"      
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <- sells$pps[2]
        s1$basis <- s1$sold_cf_basis
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
        
        ## must also update the buys to match that sells c/f (this will allow for loop2 to begin 
        ## with both the buy and sell processing the c/f (example when sell cf = 900, then the 
        ## buy row needs to be that same 900 .... do not delete, just update here and now 
        
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- TRUE
        
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0
        
        test2 <- buys
        test2 <- test2 %>% add_row(b1, .before = 1)
        buys <- test2
                      
        }

    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ], dim of buys increases with c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... dim of sells increases with c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_buysell_withcf. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
return_list <- update_buysell_withcf(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)



 There is Buy carryforward. Update buys  0 
the s2cf_quan is:  1000 
the s2cf_pps is  6.68255 
printing updated buys[1 ], dim of buys increases with c/f  


[1] 24 23

[1] 19 23

In [25]:
t(b)

,1
index,1
date,2022-12-31
type,Buy
co,AMC
quan,1050
pps,4.07
basis,4273.5
sale_quan,0
flag,FALSE
buy_cf_quan,0


In [26]:
range(x <- sort(round(stats::rnorm(10) - 1.2, 1)))

if(any(x < 0)) cat("x contains negative values\n")

[1] -2.6  0.9

x contains negative values


In [27]:
if(any(buys[1,] == TRUE)) cat("buys contains cf \n")

buys contains cf 


In [44]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE
1,2022-12-31,Buy CF,AMC,0,0.00,0.0,0,FALSE,0,⋯,0.00,0,0,5.5124,0.0,0.0,0,0,0,FALSE


# <font color = cyan> Here is another issue: when b == c, 

#### <font color = yellow>5. Snip(), delete row from buys and sells / replace row when c/f 

In [29]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    # parameters - rep the input vars (arguments)  ... may be extraneous
    b1 <- buys [1, ]
    s1 <- sells [1, ]
       
    # snip .... if 

    if(buys$type[1] == "Buy CF")   {         
        cat(" Contains buy carryforward - remove buys c/f row and DO snip sells \n")         
         # remove the original buys and sells
         buys <- buys[-2, ]    
         sells <- sells[-2, ]
        return(list(b1, s1, buys, sells))                   
        
    } else if (sells$type[1] == "Sell CF") {
        cat("sells contains carryforward .. Replace sells and DO snip buys \n") 
        sells <- sells[-2,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))    
    } else {
        cat("There is no carryforward in buys or sells, DO snip buys and Do snip sells \n")
        buys <- buys[-1,]
        sells <- sells[-1,]
        return(list(b1, s1, buys, sells))    
    }
}    
   
# fct call passing 2 arguments
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
# s1

 Contains buy carryforward - remove buys c/f row and DO snip sells 


[1] 23 23

[1] 18 23

In [30]:
t(updated_b)

,1
index,1
date,2022-12-31
type,Buy
co,AMC
quan,1050
pps,4.07
basis,4273.5
sale_quan,1000
flag,TRUE
buy_cf_quan,50


#### <font color = lime> QC: Loop 1 review buys & sells

In [31]:
buys[1:3,]
sells[1:3,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,0,0.000,0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,5,2023-01-23,Buy,AMC,1000,5.788,5788,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,AMC,0,5.5124,5.5124,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,5.5124,5512.4000,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.2550,6255.0000,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = lime> QC: Loop 1 review sale_record for accuracy

In [32]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE


#### <font color = lime> QC: Loop 1 review updated_b for accuracy

In [33]:
updated_b

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE


### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [34]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} 

sale_records exists:      


In [35]:
# stopps #  Old code

# <font color = cyan> Proof of Concept above. Make it loop 2 x.

#### <font color = yellow>1. Create data & dfs

In [36]:
b <- buys[1,]
s <- sells[1,]
b
s

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,0,0,0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,AMC,0,5.5124,5.5124,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,0


In [37]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE


### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [38]:
updated_b <- update_b()
updated_b  
    

update fct(), b$quan === s$quan ... No NEW carryforward. flag & true_false = FALSE  0 


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy CF,AMC,0,0,0,0,FALSE,0,⋯,0,0,0,5.5124,0,0,0,0,0,FALSE


In [39]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE


### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [40]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}


sale_records exists:      


In [41]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,TRUE,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,FALSE
1,2022-12-31,Buy CF,AMC,0,0.00,0.0,0,FALSE,0,⋯,0.00,0,0,5.5124,0.0,0.0,0,0,0,FALSE


#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction

In [42]:
# fct call
return_list <- update_buysell_withcf(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)



 The buy == sell, t/f No update to buys or sells.  NO carryforward.  0 
If msg here, there is no CF and potential issue exists in fct update_buysell_withcf.  


[1] 23 23

[1] 18 23

In [43]:
stopps

ERROR: Error in eval(expr, envir, enclos): object 'stopps' not found


#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [ ]:
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [ ]:
sale_records

#### <font color = lime> QC: Loop 2 review buys & sells

In [ ]:
dim(buys)
dim(sells)
head(buys,3)
head(sells,3)

In [ ]:
# stopps

# <font color = cyan> Proof of Concept above. Make it loop 3 x.

In [ ]:
# fct call 1
b <- buys[1,]
s <- sells[1,]

# fct call 2
updated_b <- update_b()
updated_b  

# fct call 3
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}

# fct call 4
return_list <- update_buysell_withcf(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]



# fct call 5
copy_snip_list <- copy_snip(buys, sells) 
b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]


In [ ]:
# fct call 1
b <- buys[1,]
s <- sells[1,]
b
s

In [ ]:
# fct call 2
updated_b <- update_b()
updated_b  

In [ ]:
# fct call 3
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}

In [ ]:
# fct call 4
return_list <- update_buysell_withcf(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]

In [ ]:
# fct call 5
copy_snip_list <- copy_snip(buys, sells) 
b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

#### <font color = lime> QC: Loop 3 review buys & sells
    


In [ ]:
dim(buys)
dim(sells)
head(buys, 5)
head(sells, 5)

In [ ]:
sale_records

In [ ]:
t(sale_records)

In [ ]:
#stopps

# <font color = cyan> Proof of Concept above. Make it loop 4 x

In [ ]:
# fct call 1
b <- buys[1,]
s <- sells[1,]

# fct call 2
updated_b <- update_b()
updated_b  

# fct call 3
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}

# fct call 4
return_list <- update_buysell_withcf(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]



# fct call 5
copy_snip_list <- copy_snip(buys, sells) 
b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]


In [ ]:
# stopps

# <font color = cyan> Proof of Concept above. Make it loop 5 x.

In [ ]:
# fct call 1
b <- buys[1,]
s <- sells[1,]

# fct call 2
updated_b <- update_b()
updated_b  

# fct call 3
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}

# fct call 4
return_list <- update_buysell_withcf(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]



# fct call 5
copy_snip_list <- copy_snip(buys, sells) 
b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]


In [ ]:
sale_records